In [74]:
import pandas as pd
import numpy as np

In [75]:
player_data = pd.read_csv("ortg and drtg stats.csv")
player_salary = pd.read_csv("Player Payment.csv", usecols=["Player", "2022-23"])
combine_player_data = player_data.merge(player_salary,how = "left", on = "Player")

# Drop all rows where with nan, inf, or -inf values
combine_player_data = combine_player_data.replace([np.inf, -np.inf], np.nan)
combine_player_data = combine_player_data.dropna()

combine_player_data['Pos'].replace(['PG', 'SG-PG', 'SG', 'SF-SG', 'SF', 'PF-SF', 'PF', 'C'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)

combine_player_data.set_index("Player", inplace=True)
combine_player_data

,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,DIFF,2022-23
Player,,,,,,,,,,,,,,,,,,,,,
Precious Achiuwa,1,7,23,TOR,55,12,1140,8.5,17.5,0.485,...,2.2,1.3,1.3,2.6,4.4,22.0,112.0,113,-1,2840160.0
Steven Adams,2,7,29,MEM,42,42,1133,6.6,11.0,0.597,...,4.1,1.5,1.9,3.3,4.1,15.1,118.0,108,10,17926829.0
Bam Adebayo,3,7,25,MIA,75,75,2598,11.6,21.4,0.540,...,4.6,1.7,1.2,3.6,4.0,29.3,115.0,111,4,30351780.0
Ochai Agbaji,4,2,22,UTA,59,22,1209,6.5,15.3,0.427,...,2.6,0.6,0.6,1.6,3.9,18.5,115.0,121,-6,3918360.0
Santi Aldama,5,6,22,MEM,77,20,1682,7.0,14.8,0.470,...,2.7,1.3,1.4,1.7,4.0,19.7,120.0,111,9,2094120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lonnie Walker IV,496,2,24,LAL,56,32,1297,8.6,19.3,0.448,...,2.3,1.1,0.6,1.7,2.6,23.9,111.0,117,-6,6479000.0
John Wall,497,0,32,LAC,34,3,755,9.0,21.9,0.408,...,11.5,1.8,0.8,5.2,3.8,25.0,102.0,115,-13,47345760.0
John Wall,497,0,32,LAC,34,3,755,9.0,21.9,0.408,...,11.5,1.8,0.8,5.2,3.8,25.0,102.0,115,-13,47345760.0


In [76]:
# Use KNN to find the nearest neighbor to a certain player
from sklearn.neighbors import NearestNeighbors

# Get the name of columns that are float or int
numerical_columns = combine_player_data.select_dtypes(include=['float64', 'int64']).columns
combine_player_data = combine_player_data[numerical_columns]

# drop evry attribute beside ORtg and DRtg
combine_player_data = combine_player_data.drop(["Age", "PTS", "Rk", "MP", "GS", "FG", "FGA", "FG%", "3P", "3PA", "3P%", "2P", "2PA", "2P%", "FT", "FTA", "FT%", "ORB", "DRB", "TRB", "AST", "STL", "BLK", "TOV", "PF", "DIFF", "2022-23"], axis=1)

def find_nearest_neighbor(player, num_neighbors=5):
    # Create a model with 2 neighbors
    model = NearestNeighbors(n_neighbors=num_neighbors+1)

    # Fit the model with the numerical columns
    model.fit(combine_player_data)

    # Find the nearest neighbor to a player
    player_index = combine_player_data.index.get_loc(player)
    player_stats = combine_player_data.iloc[player_index].values.reshape(1, -1)
    distances, indices = model.kneighbors(player_stats)

    # Convert indices to a list of names in order
    names = []
    for index in indices[0]:
        names.append(combine_player_data.iloc[index].name)
    print(names)
    return names
    


In [83]:
# Show the rows for "Shai Gilgeous-Alexander" and his nearest neighbor
player = "Shai Gilgeous-Alexander"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Shai Gilgeous-Alexander', 'Malcolm Brogdon', 'Jalen Brunson', 'Grayson Allen', 'Mike Conley', 'Quentin Grimes']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Shai Gilgeous-Alexander,0,68,125.0,113,30913750.0
1,Malcolm Brogdon,0,67,122.0,113,22600000.0
2,Jalen Brunson,0,68,125.0,117,27733332.0
3,Grayson Allen,2,72,123.0,114,8500000.0
4,Mike Conley,0,67,125.0,118,22680000.0
5,Quentin Grimes,2,71,125.0,117,2277000.0


In [84]:
# Show the rows for "Austin Reaves" and his nearest neighbor
player = "Austin Reaves"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Austin Reaves', 'Monte Morris', 'Lauri Markkanen', 'Luke Kennard', 'Mike Conley', 'Jalen Brunson']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Austin Reaves,2,64,129.0,117,1563518.0
1,Monte Morris,0,62,125.0,118,9125000.0
2,Lauri Markkanen,4,66,125.0,116,16475454.0
3,Luke Kennard,2,59,128.0,116,14415545.0
4,Mike Conley,0,67,125.0,118,22680000.0
5,Jalen Brunson,0,68,125.0,117,27733332.0


In [85]:
# Show the rows for "Jimmy Butler" and his nearest neighbor
player = "Jimmy Butler"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Jimmy Butler', 'Larry Nance Jr.', 'Clint Capela', 'Al Horford', 'Nick Richards', 'Thomas Bryant']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Jimmy Butler,4,64,135.0,112,37653300.0
1,Larry Nance Jr.,6,65,134.0,110,9672727.0
2,Clint Capela,7,65,136.0,112,18706896.0
3,Al Horford,7,63,133.0,112,26500000.0
4,Nick Richards,7,65,130.0,113,1782621.0
5,Thomas Bryant,7,59,132.0,112,1836090.0


In [86]:
# Show the rows for "Larry Nance Jr." and his nearest neighbor
player = "Larry Nance Jr."
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Larry Nance Jr.', 'Clint Capela', 'Al Horford', 'Jimmy Butler', 'Jarrett Allen', 'Nick Richards']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Larry Nance Jr.,6,65,134.0,110,9672727.0
1,Clint Capela,7,65,136.0,112,18706896.0
2,Al Horford,7,63,133.0,112,26500000.0
3,Jimmy Butler,4,64,135.0,112,37653300.0
4,Jarrett Allen,7,68,132.0,108,20000000.0
5,Nick Richards,7,65,130.0,113,1782621.0


In [87]:
# Show the rows for "Kevon Looney" and his nearest neighbor
player = "Kevon Looney"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Kevon Looney', 'Dwight Powell', 'Walker Kessler', 'Onyeka Okongwu', 'Luke Kornet', 'Domantas Sabonis']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Kevon Looney,7,82,147.0,111,8000000.0
1,Dwight Powell,7,76,140.0,116,11080125.0
2,Walker Kessler,7,74,140.0,111,2696400.0
3,Onyeka Okongwu,7,80,134.0,112,6395160.0
4,Luke Kornet,7,69,143.0,111,2133278.0
5,Domantas Sabonis,7,79,132.0,114,18500000.0


In [88]:
# Show the rows for "Malcolm Brogdon" and his nearest neighbor
player = "Malcolm Brogdon"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Malcolm Brogdon', 'Shai Gilgeous-Alexander', 'Jrue Holiday', 'Darius Garland', 'Donovan Mitchell', 'Jalen Brunson']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Malcolm Brogdon,0,67,122.0,113,22600000.0
1,Shai Gilgeous-Alexander,0,68,125.0,113,30913750.0
2,Jrue Holiday,0,67,118.0,112,33665040.0
3,Darius Garland,0,69,118.0,113,8920795.0
4,Donovan Mitchell,2,68,118.0,111,30913750.0
5,Jalen Brunson,0,68,125.0,117,27733332.0


In [89]:
# Show the rows for "Grayson Allen" and his nearest neighbor
player = "Grayson Allen"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Grayson Allen', 'Isaiah Joe', 'Quentin Grimes', 'Isaac Okoro', 'Shai Gilgeous-Alexander', 'Jakob Poeltl']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Grayson Allen,2,72,123.0,114,8500000.0
1,Isaiah Joe,2,73,125.0,116,1836090.0
2,Quentin Grimes,2,71,125.0,117,2277000.0
3,Isaac Okoro,4,76,123.0,112,7040880.0
4,Shai Gilgeous-Alexander,0,68,125.0,113,30913750.0
5,Jakob Poeltl,7,72,123.0,114,9398148.0


In [90]:
# Show the rows for "Trey Murphy III" and his nearest neighbor
player = "Trey Murphy III"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Trey Murphy III', 'Drew Eubanks', 'Sam Hauser', 'Josh Hart', 'Domantas Sabonis', 'Chris Boucher']


,Player,Pos,G,ORtg,DRtg,2022-23
0,Trey Murphy III,4,79,128.0,114,3206520.0
1,Drew Eubanks,7,78,130.0,115,1836090.0
2,Sam Hauser,6,80,124.0,113,1637966.0
3,Josh Hart,3,76,125.0,116,12960000.0
4,Domantas Sabonis,7,79,132.0,114,18500000.0
5,Chris Boucher,6,76,125.0,112,12690000.0


In [91]:
# Show the rows for "Xavier Tillman Sr." and his nearest neighbor
player = "Xavier Tillman Sr."
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Xavier Tillman Sr.', 'Thomas Bryant', 'Brandon Clarke', 'Al Horford', 'Nick Richards', 'Larry Nance Jr.']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Xavier Tillman Sr.,6,61,129.0,109,1782621.0
1,Thomas Bryant,7,59,132.0,112,1836090.0
2,Brandon Clarke,6,56,131.0,109,4343920.0
3,Al Horford,7,63,133.0,112,26500000.0
4,Nick Richards,7,65,130.0,113,1782621.0
5,Larry Nance Jr.,6,65,134.0,110,9672727.0


In [92]:
# Show the rows for "Trey Murphy III" and his nearest neighbor
player = "Trey Murphy III"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Trey Murphy III', 'Drew Eubanks', 'Sam Hauser', 'Josh Hart', 'Domantas Sabonis', 'Chris Boucher']


,Player,Pos,G,ORtg,DRtg,2022-23
0,Trey Murphy III,4,79,128.0,114,3206520.0
1,Drew Eubanks,7,78,130.0,115,1836090.0
2,Sam Hauser,6,80,124.0,113,1637966.0
3,Josh Hart,3,76,125.0,116,12960000.0
4,Domantas Sabonis,7,79,132.0,114,18500000.0
5,Chris Boucher,6,76,125.0,112,12690000.0


In [93]:
# Show the rows for "Luke Kornet" and his nearest neighbor
player = "Luke Kornet"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Luke Kornet', 'Walker Kessler', 'Clint Capela', 'Dwight Powell', 'Jimmy Butler', 'Larry Nance Jr.']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Luke Kornet,7,69,143.0,111,2133278.0
1,Walker Kessler,7,74,140.0,111,2696400.0
2,Clint Capela,7,65,136.0,112,18706896.0
3,Dwight Powell,7,76,140.0,116,11080125.0
4,Jimmy Butler,4,64,135.0,112,37653300.0
5,Larry Nance Jr.,6,65,134.0,110,9672727.0


In [94]:
# Show the rows for "Tyus Jones" and his nearest neighbor
player = "Tyus Jones"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Tyus Jones', 'Immanuel Quickley', 'Terance Mann', 'Isaac Okoro', 'Sam Hauser', 'Keegan Murray']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Tyus Jones,0,80,122.0,113,15000000.0
1,Immanuel Quickley,0,81,122.0,115,2316240.0
2,Terance Mann,2,81,123.0,117,1930681.0
3,Isaac Okoro,4,76,123.0,112,7040880.0
4,Sam Hauser,6,80,124.0,113,1637966.0
5,Keegan Murray,4,80,119.0,117,8008440.0


In [95]:
# Show the rows for "Isaiah Joe" and his nearest neighbor
player = "Isaiah Joe"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Isaiah Joe', 'Quentin Grimes', 'Grayson Allen', 'Josh Hart', 'Corey Kispert', 'Jalen Brunson']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Isaiah Joe,2,73,125.0,116,1836090.0
1,Quentin Grimes,2,71,125.0,117,2277000.0
2,Grayson Allen,2,72,123.0,114,8500000.0
3,Josh Hart,3,76,125.0,116,12960000.0
4,Corey Kispert,4,74,124.0,120,3552840.0
5,Jalen Brunson,0,68,125.0,117,27733332.0


In [96]:
# Show the rows for "Isaac Okoro" and his nearest neighbor
player = "Isaac Okoro"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Isaac Okoro', 'Jarred Vanderbilt', 'Chris Boucher', 'Santi Aldama', 'Nicolas Batum', 'Nicolas Batum']


,Player,Pos,G,ORtg,DRtg,2022-23
0,Isaac Okoro,4,76,123.0,112,7040880.0
1,Jarred Vanderbilt,6,78,123.0,112,4374000.0
2,Chris Boucher,6,76,125.0,112,12690000.0
3,Santi Aldama,6,77,120.0,111,2094120.0
4,Nicolas Batum,6,78,120.0,114,19700319.0
5,Nicolas Batum,6,78,120.0,114,19700319.0
6,Nicolas Batum,6,78,120.0,114,19700319.0
7,Nicolas Batum,6,78,120.0,114,19700319.0
8,Nicolas Batum,6,78,120.0,114,19700319.0
9,Nicolas Batum,6,78,120.0,114,19700319.0


In [97]:
# Show the rows for "Chris Boucher" and his nearest neighbor
player = "Chris Boucher"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Chris Boucher', 'Isaac Okoro', 'Jarred Vanderbilt', 'Sam Hauser', 'Brook Lopez', 'Jakob Poeltl']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Chris Boucher,6,76,125.0,112,12690000.0
1,Isaac Okoro,4,76,123.0,112,7040880.0
2,Jarred Vanderbilt,6,78,123.0,112,4374000.0
3,Sam Hauser,6,80,124.0,113,1637966.0
4,Brook Lopez,7,78,122.0,109,13906976.0
5,Jakob Poeltl,7,72,123.0,114,9398148.0


In [98]:
# Show the rows for "Walker Kessler" and his nearest neighbor
player = "Walker Kessler"
num = 5

closest = find_nearest_neighbor(player, num)

# Print the data for the closest player
combine_player_data.loc[closest].merge(player_salary,how = "left", on = "Player")

['Walker Kessler', 'Dwight Powell', 'Luke Kornet', 'Onyeka Okongwu', 'Nic Claxton', 'Domantas Sabonis']


/Users/howard/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Player,Pos,G,ORtg,DRtg,2022-23
0,Walker Kessler,7,74,140.0,111,2696400.0
1,Dwight Powell,7,76,140.0,116,11080125.0
2,Luke Kornet,7,69,143.0,111,2133278.0
3,Onyeka Okongwu,7,80,134.0,112,6395160.0
4,Nic Claxton,7,76,132.0,108,8500000.0
5,Domantas Sabonis,7,79,132.0,114,18500000.0
